In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, InputLayer, Embedding, Conv1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter

In [0]:
from numpy.random import seed
seed(0)
from tensorflow.random import set_seed
set_seed(0)

In [0]:
def visualize(model):
  plt.plot(model.history.history['accuracy'])
  plt.plot(model.history.history['val_accuracy'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

In [34]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [0]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()


In [36]:
tagged_sentences[0]


[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

In [0]:
sentences, sentence_tags =[], [] 
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(sentence)
    sentence_tags.append(tags)

In [0]:
sent_train, sent_test, tag_train, tag_test = train_test_split(sentences, sentence_tags, test_size=0.2, random_state=0)


In [0]:
vocab = Counter()
for sent in sent_train:
    sent = [word.lower() for word in sent]
    vocab.update(sent)

In [0]:
filtered_vocab = {word for word in vocab if vocab[word] > 5}


In [0]:
word2id = {'PAD':0,'UNK':1}    
for i,word in enumerate(filtered_vocab):
      word2id[word] = i + 2

id2word = {i:word for word, i in word2id.items()}

In [0]:
tag2id = {'PAD':0}  
for tags in tag_train:
    for tag in tags:
      if tag.lower() not in tag2id:
        tag2id[tag.lower()] = len(tag2id)

id2tag = {i:tag for tag, i in tag2id.items()}

In [0]:
def data2ints(data, smth2id):
  int_data = []
  for seq in data:
      int_seq = []
      for i in seq:
          try:
            int_seq.append(smth2id[i.lower()])
          except KeyError:
            int_seq.append(smth2id['UNK'])
  
      int_data.append(int_seq)
  return int_data

In [44]:
X_train_ids, X_test_ids = data2ints(sent_train, word2id), data2ints(sent_test, word2id)
y_train_ids, y_test_ids = data2ints(tag_train, tag2id), data2ints(tag_test, tag2id)


print(X_train_ids[0])
print(X_test_ids[0])
print(y_train_ids[0])
print(y_test_ids[0])

[264, 1, 1, 1, 1594, 1, 309, 1218, 519]
[401, 12, 1190, 1069, 317, 592, 113, 1, 1040, 1, 1605, 449, 590, 519]
[1, 1, 1, 2, 1, 1, 3, 4, 5]
[18, 19, 21, 24, 10, 25, 24, 18, 21, 14, 3, 7, 15, 5]


In [45]:
MAX_LEN = max(len(x) for x in sent_train)
MAX_LEN

128

In [0]:
X_train_word, X_test_word = pad_sequences(X_train_ids, maxlen=MAX_LEN, padding='post'), pad_sequences(X_test_ids, maxlen=MAX_LEN, padding='post')
y_train_pad, y_test_pad = pad_sequences(y_train_ids, maxlen=MAX_LEN, padding='post'), pad_sequences(y_test_ids, maxlen=MAX_LEN, padding='post')

In [0]:
y_train_word, y_test_word = to_categorical(y_train_pad, num_classes=len(tag2id)), to_categorical(y_test_pad, num_classes=len(tag2id))


In [48]:
print(X_train_word.shape, y_train_word.shape, X_test_word.shape, y_test_word.shape)


(3131, 128) (3131, 128, 47) (783, 128) (783, 128, 47)


# Мое решение

CharCNN-(bi)LSTM for sequence tagging

как мы видим, задача pos-тэггинга плохо решается без морфологической информации . Ваша задача перестроить  нашу сеть так, чтобы она учитывала эту информацию. Постройте сеть, которая имела бы два input слоя:

1. один такой же, как мы сделали, подает слова в embedding-слой, а поверх него работает (bi)lstm, проходящая по всей последовательности
2. второй входной слой должен брать последовательность слов, каждое из которых представлено буквами (используйте паддинг для добивания каждого слова до максимальной длины слова). Таким образом второй input слой будет принимать массивы размера (batch_size, max_seq_len, max_char_len). Данные с этого input слоя должны попадать в embedding слой (на выходе (batch_size, max_seq_len, max_char_len, char_emb)), затем к ним должна применяться свертка (на выходе (batch_size, max_seq_len, ~max_char_len, kernel_size)), затем расплющивание (на выходе (batch_size, max_seq_len, ~max_char_len*kernel_size)).
 NB: чтобы слои Conv1D и Flatten возвращали указанные размерности, они должны работать на элементах последовательности, а не на всей последовательности сразу. Для этого вам понадобится обернуть их в TimeDistributed.

Полученные выходы с 1. и 2. сконкатенируйте, и пройдитесь по ним (bi)LSTM. В конце, как и в построенной нами сети, испольуйте TimeDistributed Dense слой.

Для того, чтобы можно было глазами оценить результат, напишите ф-цию, которая бы брала на вход английское предложение, например любое предложение из теста, и выводила предсказания сети в виде тэгов.

Если с построением сети совсем сложно и непонятно, вам должен помочь код вот здесь https://www.depends-on-the-definition.com/lstm-with-char-embeddings-for-ner/  (эмбеддинги символов обрабатываются не свертками, но идея очень похожая)

In [0]:
chars = set([w_i for sentence in sentences for word in sentence for w_i in word])

In [50]:
n_chars = len(chars)
print(n_chars)

79


In [51]:
MAX_LEN_CHAR = 0
for sentence in sentences:
  for w in sentence:
    if MAX_LEN_CHAR < len(w):
      MAX_LEN_CHAR = len(w)
MAX_LEN_CHAR

24

In [52]:
MAX_LEN = max(len(x) for x in sent_train)
MAX_LEN

128

In [0]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [0]:
def char2ints(data, char2idx):
  X_char = []
  for sentence in sentences:
      sent_seq = []
      for i in range(MAX_LEN):
          word_seq = []
          for j in range(MAX_LEN_CHAR):
              try:
                  word_seq.append(char2idx.get(sentence[i][0][j]))
              except:
                  word_seq.append(char2idx.get("PAD"))
          sent_seq.append(word_seq)
      X_char.append(np.array(sent_seq))
      
  return X_char

In [0]:
X_train_ids_char, X_test_ids_char = char2ints(sent_train, char2idx), char2ints(sent_test, char2idx)


In [0]:
y_train_ids_char, y_test_ids_char = data2ints(tag_train, tag2id), data2ints(tag_test, tag2id)

In [0]:
X_train_char, X_test_char = pad_sequences(X_train_ids_char, maxlen=MAX_LEN, padding='post'), pad_sequences(X_test_ids_char, maxlen=MAX_LEN, padding='post')
y_train_pad, y_test_pad = pad_sequences(y_train_ids_char, maxlen=MAX_LEN, padding='post'), pad_sequences(y_test_ids_char, maxlen=MAX_LEN, padding='post')

In [59]:
print(X_train_word.shape, y_train_word.shape, X_test_word.shape, y_test_word.shape, X_train_char.shape, X_test_char.shape)

(3131, 128) (3131, 128, 47) (783, 128) (783, 128, 47) (3914, 128, 24) (3914, 128, 24)


In [0]:
input_word = tf.keras.layers.Input(shape=(MAX_LEN,))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, mask_zero=True)(input_word)
lstm1 = Bidirectional(LSTM(256, return_sequences=True))(emb_word)

input_char = tf.keras.layers.Input(shape=(MAX_LEN, MAX_LEN_CHAR,))
emb_char = Embedding(input_dim=len(char2idx), output_dim=50)(input_char)
conv1 = TimeDistributed(Conv1D(kernel_size=12, filters=4, strides=1))(emb_char)
flat1 = TimeDistributed(tf.keras.layers.Flatten())(conv1)

concat = tf.keras.layers.concatenate([lstm1, flat1])
main_lstm = Bidirectional(LSTM(256, return_sequences=True))(concat)
dense = TimeDistributed(Dense(len(tag2id), activation="sigmoid"))(main_lstm)

model = tf.keras.Model([input_word, input_char], dense)

In [70]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 128, 24)]    0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding_11 (Embedding)        (None, 128, 24, 50)  4050        input_12[0][0]                   
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 128, 100)     168100      input_11[0][0]                   
______________________________________________________________________________________________

In [72]:
model.fit(X_train_word, y_train_word, validation_data=(X_test_word, y_test_word), batch_size=128, epochs=10)

Epoch 1/10


AssertionError: ignored